In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import numpy as np

import ipywidgets as widgets
from ipywidgets import HBox, Layout, VBox, GridBox, IntSlider, Checkbox, interactive_output

# Set the universal font type to times new roman
plt.rcParams["font.family"] = "Times New Roman"

In [6]:
widths = "230px"
step1 = 50

V1 = IntSlider(min=1000, max=2300, value=1500, step=step1, 
               description="V1", layout=Layout(width=widths))
P1 = IntSlider(min=1800, max=2200, value=2000, step=step1, 
               description="P1", layout=Layout(width=widths))
h1 = IntSlider(min=0, max=20, value=18, description="h1", layout=Layout(width=widths))

V2 = IntSlider(min=2000, max=2800, value=2500, step=step1, 
               description="V2", layout=Layout(width=widths))
P2 = IntSlider(min=2300, max=2600, value=2400, step=step1, 
               description="P2", layout=Layout(width=widths))
h2 = IntSlider(min=5, max=20, value=10, description="h2", layout=Layout(width=widths))

V3 = IntSlider(min=2500, max=3000, value=2700, step=step1, 
               description="V3", layout=Layout(width=widths))
P3 = IntSlider(min=2400, max=2900, value=2500, step=step1, 
               description="P3", layout=Layout(width=widths))
h3 = IntSlider(min=5, max=20, value=12, description="h3", layout=Layout(width=widths))
show_lp = Checkbox(description='Show Layers', value=True)
show_rc = Checkbox(description='Show Coeffs.', value=False)
show_ry = Checkbox(description='Show Ray path', value=False)

# Create a Layout object specifying a 3x3 grid
grid_layout = Layout(border='solid 1px black',
        margin='0px 0px 0px',
        padding='0px 0px 0px 0px',
        grid_gap="10px",
        grid_template_columns="repeat(3, 25%)")#,width='500px'

widget_grid = GridBox(children=[V1, P1, h1, V2, P2, h2, V3, P3, h3, show_lp, show_rc, show_ry],
                 layout=grid_layout)

def refl_coef(l1,l2):
    """
    Calculates reflection and transmission coefficients

    Args:
        l1 (float): acoustic impedance layer 1
        l2 (float): acoustic impedance layer 2

    Returns:
        (rc,tc) (tuple): (reflection, transmission coefficent)
    """
    rc = (l2-l1)/(l2+l1)
    tc = 1 - rc
    return rc,tc

def trans_coef(l1:float,l2:float)->float:
    """
    Calculates only transmission coefficients

    Args:
        l1 (float): acoustic impedance layer 1
        l2 (float): acoustic impedance layer 2

    Returns:
        tc (float): transmission coefficent
    """
    tc = (2*l1)/(l2+l1)
    return tc

def raypath_length_two_way(src_l:float,rcv_l:float,h:float)->float:
    """
    Calculate two-way ray path length
        In this notebook, the unit is meters

    Args:
        src_l (float): source x-location
        rcv_l (float): receiver x-location
        h (float): layer thickness

    Returns:
        rypth_l (float): two-way ray path length
    """
    rypth_l = np.sqrt((h**2)+(((rcv_l-src_l)/2)**2)) * 2
    return np.around(rypth_l, 2)

def twt_msec(rpl:float,vl:float)->float:
    """
    Calculate two-way-travel time

    Args:
        rpl (float): two-way raypath length in m
        vl (float): layer velocity in meter/s

    Returns:
        twt (float): two-way-time in millisecs
    """
    twt = (rpl/vl) * 1000
    return np.around(twt, 2)

# Define the plotter function
def plot_function(V1,P1,h1,V2,P2,h2,V3,P3,h3,show_lp,show_rc,show_ry):
    x0, pwdth, txst = -5, 60, 33         # x-start-plot, x-width-plot, x-start-text-labels
    ms, fs, lw = 200, 13, 0.5
    f1,f2,f3 = 'whitesmoke','gainsboro','silver'
    l1_top = 0
    l1_base = l1_top+h1
    l2_top = l1_base
    l2_base = l2_top+h2
    l3_top = l2_base
    l3_base = l3_top+h3

    # Calculate Acoustic Impedance,
    I1,I2,I3 = V1*P1, V2*P2, V3*P3
    rc_12,tc_12 = refl_coef(I1,I2)
    rc_23,tc_23 = refl_coef(I2,I3)

    # Define the sources and receivers
    src,rcv = 0, np.arange(10, 60, 10)
    all_sr = np.insert(rcv, 0, src)

    # Calculate the normal moveout
    rpl_1 = raypath_length_two_way(src, all_sr, h1)
    twt_1 = twt_msec(rpl_1, V1)
    nmo_1 = twt_1 - twt_1[0]

    fig,ax = plt.subplots(1,2,figsize=(10,4), 
                          gridspec_kw={'width_ratios':[2,1], 'wspace':0.05})
    
    ###################################### Left Subplot ######################################
    # -------------------------- Plot the Model Layers
    ax[0].add_patch(Rectangle((x0, l1_top), pwdth, h1, ec = 'k', fc = f1, lw=lw)) # Layer 1
    ax[0].add_patch(Rectangle((x0, l2_top), pwdth, h2, ec = 'k', fc = f2, lw=lw)) # Layer 2
    ax[0].add_patch(Rectangle((x0, l3_top), pwdth, h3, ec = 'k', fc = f3, lw=lw)) # Layer 3
    # -------------------------- Label the Layer Boundaries
    ax[0].text(x0+1, l1_base-1, "Boundary A", size=fs)
    ax[0].text(x0+1, l2_base-1, "Boundary B", size=fs)
    # -------------------------- Plot and label the source and receivers
    ax[0].scatter(src,l1_top, marker="*", s=ms, ec='k', c='k')
    ax[0].scatter(rcv,np.ones_like(rcv)*(l1_top-1), marker="^", s=ms/2, ec='k', c='w')
    for xi in all_sr:
        ax[0].text(xi-1,l1_top-3,xi,size=fs-2)
    # -------------------------- Plot the Velocity, density, acoustic impedance
    if show_lp:
        rho1,rho2,rho3 = r'$\rho_{1}$',r'$\rho_{2}$',r'$\rho_{3}$'
        ax[0].text(txst, l1_base-3, f"{r'$V_{1}$'}={V1} m/s | {rho1}={P1} {r'$kg/m^{3}$'}")
        ax[0].text(txst, l2_base-3, f"{r'$V_{2}$'}={V2} m/s | {rho2}={P2} {r'$kg/m^{3}$'}")
        ax[0].text(txst, l3_base-3, f"{r'$V_{3}$'}={V3} m/s | {rho3}={P3} {r'$kg/m^{3}$'}")
        ax[0].text(txst, l1_base-1, f"{r'$I_{1}$'} = {I1:.1e}", c='b')
        ax[0].text(txst, l2_base-1, f"{r'$I_{2}$'} = {I2:.1e}", c='b')
        ax[0].text(txst, l3_base-1, f"{r'$I_{3}$'} = {I3:.1e}", c='b')
    # -------------------------- Refl. Coef, and Trans. Coeff
    if show_rc:
        ax[0].text(txst-15, l1_base-1, f"{r'$C_{R1}$'} = {rc_12:.2f}", c='deeppink')
        ax[0].text(txst-15, l2_top+2, f"{r'$C_{T1}$'} = {tc_12:.2f}", c='deeppink')
        ax[0].text(txst-15, l2_base-1, f"{r'$C_{R2}$'} = {rc_23:.2f}", c='deeppink')
        ax[0].text(txst-15, l3_top+2, f"{r'$C_{T2}$'} = {tc_23:.2f}", c='deeppink')
    # -------------------------- Plot the raypaths on the model
    if show_ry:
        for xi in all_sr:
            mid = (xi - src)/2
            ray_x = [src, mid, xi]
            ray_y = [l1_top, l1_base, l1_top]
            ax[0].plot(ray_x,ray_y,lw=0.5,c='r')
    

    ax[0].set_xlim(x0, x0+pwdth),ax[0].set_ylim(l3_base, l1_top-5),
    ax[0].set_xlabel("Distance (m)",size=fs),ax[0].set_ylabel("Depth (m)",size=fs)

    ###################################### Right Subplot ######################################
    ax[1].set_title("Two-way-travel time",size=fs)
    
    # -------------------------- Plot the normal moveout
    if show_ry:
        for idx,xi in enumerate(all_sr[1:]):
            ax[1].text(xi-3,twt_1[0]-2,f"{nmo_1[idx+1]:.1f}",size=fs-2)
        ax[1].text(all_sr[0]-1, twt_1[0]-2, r"$t_{0}$", size=fs-2)
        ax[1].scatter(all_sr, twt_1, marker="o", c="k", s=ms/10)
    ax[1].set_xlim(x0, x0+pwdth), ax[1].set_ylim(twt_1[-1]+5, -3)
    ax[1].yaxis.tick_right(),ax[1].yaxis.set_label_position("right")
    ax[1].set_xlabel("Distance (m)",size=fs),ax[1].set_ylabel("TWT (msec)",size=fs)
plt.show()

# Assign the function parameters to the interactive sliders
output = interactive_output(plot_function,{'V1':V1,'P1':P1,'h1':h1,
                                         'V2':V2,'P2':P2,'h2':h2,'V3':V3,'P3':P3,'h3':h3,
                                         "show_lp":show_lp,'show_rc':show_rc,'show_ry':show_ry})

### Equations used
- Reflection and Transmission Coefficients
    $$
    \begin{gather*}
    C_R = \frac {I_2 - I_1} {I_2 + I_1}\\
    \ \\
    C_T = \frac {2I_1} {I_2 + I_1}\\
    \end{gather*}
    $$
    - where
        - $I_1 = \rho * V_1$
        - $I_1$ is Acoustic Impedance
        - $\rho$ is Density
        - $V_1$ is Velocity
<br><br>

- Normal Incidence Two-way-travel time
    $$t_0 = \frac {2h} {V_1}$$
    - where
        - $t_0 = \text{origin time at source}$
        - $h = \text{layer thickness}$
        - $V_1 = \text{layer 1 velocity}$

In [7]:
display(VBox([widget_grid, output]))